In [8]:
import numpy as np
from io import StringIO
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import pandas_datareader
import datetime
import pandas_datareader.data as web
from scipy.stats import norm

tickers = ['HD','VOD','ORCL','VOO']
              

data = pd.DataFrame()

for t in tickers:
    data[t] = web.DataReader(t, data_source='yahoo', start='2018-1-1')['Adj Close']
data.to_csv("GROWTHUSASEPT22.csv")

In [9]:
returns = np.log(data / data.shift(1))
vols = returns.std() * 252 ** 0.5 *100
vols

HD      28.961283
VOD     30.830822
ORCL    29.677393
VOO     21.761204
dtype: float64

In [10]:
annual_returns = returns.mean() * 252 * 100
annual_returns

HD      10.385204
VOD    -14.731825
ORCL     9.692496
VOO      9.343463
dtype: float64

In [11]:
import numpy as np
import pandas as pd
from scipy.stats import norm
import pandas_datareader.data as web
import datetime
import math

class ValueAtRiskMonteCarlo:
    
    def __init__(self, S, mu, sigma, c, n, iterations):
        self.S = S
        self.mu = mu
        self.sigma = sigma
        self.c = c
        self.n = n
        self.iterations = iterations 

    def simulation(self):
        stock_data = np.zeros([self.iterations, 1])	
        rand = np.random.normal(0, 1, [1, self.iterations])

        #equation for the S(t) stock price
        stock_price = self.S*np.exp(self.n*(self.mu - 0.5*self.sigma**2)+self.sigma*np.sqrt(self.n)*rand)
        #we have to sort the stock prices to determine the percentile
        stock_price = np.sort(stock_price)

        #it depends on the confidence level: 95% -> 5 and 99% -> 1
        percentile = np.percentile(stock_price,(1-self.c)*100)

        return self.S-percentile




In [12]:
df = pd.DataFrame(columns = ['Ticker', 'Cov_mar','Market_var','Beta','Volatility%','Return%','Sharp Ratio','CAGR','MAXDD%','Value at Risk']) 
data = pd.DataFrame()
data['^IXIC'] = web.DataReader(t, data_source='yahoo', start='2018-1-1')['Adj Close'] 

for t in tickers:
 
    data[t] = web.DataReader(t, data_source='yahoo', start='2018-1-1')['Adj Close']  

    sec_returns = np.log( data / data.shift(1) )
    cov = sec_returns.cov()*252
    cov_with_market = cov[t][0]
 
      
    dr = data[t].pct_change()
    dcum = (1 + dr).cumprod()
    n = len(data)/252
    CAGR = ((dcum[-1])**(1/n) - 1)*100
      
 
    
    droll = dcum.cummax()
    ddw = droll - dcum
    ddmax = ddw/droll
    max_dd = (ddmax.max())*100
   
 

    market_var = sec_returns['^IXIC'].var() * 252

    returns = np.log(data[t] / data[t].shift(1))
    MSFT_beta = cov_with_market / market_var
    MSFT_er = returns.mean()*252
    Sharp = (MSFT_er - 0.05) / (sec_returns[t].std() * 252 ** 0.5)
   

    #returns = np.log(data[t] / data[t].shift(1))
    vols = returns.std() * 252 ** 0.5 *100
    annual_returns = returns.mean() * 252 * 100
    
    S = 100000    #this is the investment (stocks or whatever)
    c=0.95   #condifence level: this time it is 99%
    n = 730   #1 day 
    iterations = 100000 #number of paths in the Monte-Carlo simulation
    mu = np.mean(returns)
    sigma = np.mean(returns.std())

    model = ValueAtRiskMonteCarlo(S,mu,sigma,c,n,iterations)
    val = model.simulation()

 
    
 
    df = df.append(pd.Series([t,cov_with_market,market_var,MSFT_beta,vols,annual_returns,Sharp,CAGR,max_dd,val], index=df.columns ), ignore_index=True)
                                    
                        
  

/var/folders/jx/qw4vpgyd2gxg8zhm3nm041j40000gp/T/ipykernel_12004/2650492764.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series([t,cov_with_market,market_var,MSFT_beta,vols,annual_returns,Sharp,CAGR,max_dd,val], index=df.columns ), ignore_index=True)
/var/folders/jx/qw4vpgyd2gxg8zhm3nm041j40000gp/T/ipykernel_12004/2650492764.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series([t,cov_with_market,market_var,MSFT_beta,vols,annual_returns,Sharp,CAGR,max_dd,val], index=df.columns ), ignore_index=True)
/var/folders/jx/qw4vpgyd2gxg8zhm3nm041j40000gp/T/ipykernel_12004/2650492764.py:53: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series([t,cov_with_market,market

In [13]:
df

,Ticker,Cov_mar,Market_var,Beta,Volatility%,Return%,Sharp Ratio,CAGR,MAXDD%,Value at Risk
0,HD,0.048081,0.047355,1.015341,28.961296,10.385196,0.185945,10.933922,37.993708,46761.193226
1,VOD,0.038059,0.047355,0.803698,30.830821,-14.731825,-0.640003,-13.687369,59.659984,76118.914855
2,ORCL,0.043816,0.047355,0.925276,29.677386,9.692494,0.158117,10.168779,37.733302,49285.468046
3,VOO,0.047355,0.047355,1.000000,21.761204,9.343459,0.199596,9.785243,33.993014,33279.452755


In [14]:

df1 = df[(df['CAGR']>10 ) & (df['Return%']>10)]
df1.sort_values(by='Return%', ascending=False)

,Ticker,Cov_mar,Market_var,Beta,Volatility%,Return%,Sharp Ratio,CAGR,MAXDD%,Value at Risk
0,HD,0.048081,0.047355,1.015341,28.961296,10.385196,0.185945,10.933922,37.993708,46761.193226
